In [27]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import os
import random

## Get histone metadata

In [28]:
metadata=pd.read_csv("../encode-histone/metadata.tsv",delimiter="\t")
print(metadata.shape)

(1861, 56)


In [29]:
metadata["Output type"].value_counts()

peaks               1314
stable peaks         243
replicated peaks     236
hotspots              68
Name: Output type, dtype: int64

In [30]:
metadata_stable=metadata[metadata["Output type"].isin(["stable peaks","replicated peaks"])]

In [31]:
metadata_stable_noaudits=metadata_stable[metadata_stable["Audit ERROR"].isnull()]

In [32]:
metadata_stable_noaudits.head()

,File accession,File format,File type,File format type,Output type,File assembly,Experiment accession,Assay,Biosample term id,Biosample term name,...,Assembly,Genome annotation,Platform,Controlled by,File Status,s3_uri,Audit WARNING,Audit INTERNAL_ACTION,Audit NOT_COMPLIANT,Audit ERROR
3,ENCFF243QBZ,bed narrowPeak,bed,narrowPeak,replicated peaks,hg19,ENCSR692ICP,ChIP-seq,CL:0000624,"CD4-positive, alpha-beta T cell",...,NaN,NaN,NaN,released,s3://encode-public/2018/01/20/34fb463a-0625-4e...,NaN,NaN,NaN,NaN,NaN
4,ENCFF660BIT,bed narrowPeak,bed,narrowPeak,stable peaks,hg19,ENCSR681OSD,ChIP-seq,CL:0000625,"CD8-positive, alpha-beta T cell",...,NaN,NaN,NaN,released,s3://encode-public/2018/01/21/2590a4a6-cc7b-4a...,NaN,NaN,NaN,NaN,NaN
8,ENCFF912DJF,bed narrowPeak,bed,narrowPeak,replicated peaks,hg19,ENCSR161XBV,ChIP-seq,CL:0000895,"naive thymus-derived CD4-positive, alpha-beta ...",...,NaN,NaN,NaN,released,s3://encode-public/2018/01/26/d9c2eb95-d7ff-47...,NaN,NaN,NaN,NaN,NaN
10,ENCFF277QQY,bed narrowPeak,bed,narrowPeak,stable peaks,hg19,ENCSR347HBG,ChIP-seq,NTR:0003079,fibroblast of breast,...,NaN,NaN,NaN,released,s3://encode-public/2018/01/21/8390eefe-b18e-4d...,NaN,NaN,NaN,NaN,NaN
13,ENCFF874WPQ,bed narrowPeak,bed,narrowPeak,stable peaks,hg19,ENCSR311XVL,ChIP-seq,NTR:0003079,fibroblast of breast,...,NaN,NaN,NaN,released,s3://encode-public/2018/01/21/337bb77b-8f78-4f...,NaN,NaN,NaN,NaN,NaN


In [33]:
df_exp=metadata_stable_noaudits[["Biosample term name","Experiment target"]].groupby(["Biosample term name","Experiment target"]).size().reset_index()

In [34]:
len(df_exp["Biosample term name"].unique())

52

In [35]:
len(df_exp["Experiment target"].unique())

11

In [36]:
df_exp[0].sum()

479

In [37]:
def create_combine_script(metadata):
    new_script_file = "/cellar/users/mpagadal/Data/projects/germline-immune/epigenetic/scripts/combine-bed-no-gz.sh"
    with open(new_script_file, 'w') as out_file:
        # header 
        out_file.write('#! /bin/bash\n')
        out_file.write('#SBATCH --mem=10G\n')
        out_file.write('#SBATCH -o ./out/%A.%x.%a.out # STDOUT\n')
        out_file.write('#SBATCH -e ./err/%A.%x.%a.err # STDERR\n')
        out_file.write("\n")
        # list of genes
        out_file.write("date\n")
        out_file.write("\n")
        for x in metadata["Biosample term name"].unique():
            df=metadata[metadata["Biosample term name"]==x]
            for y in df["Experiment target"].unique():
                files=df[df["Experiment target"]==y]["File accession"].tolist()
                files=["../encode-histone/"+x+".bed" for x in files]
                x=x.replace(" ","")
                x=x.replace("'","")
                x=x.replace(",","")
                y=y.split("-")[0]
                y=y.replace(" ","")
                out_file.write("cat {} > ../combined-beds/histone/{}.{}.bed".format(" ".join(files),x,y))
                out_file.write("\n")
                out_file.write("\n")
        
        out_file.write("date\n")
        #out_file.write("\n")

In [38]:
create_combine_script(metadata_stable_noaudits)

In [41]:
metadata_stable_noaudits[["Experiment accession","File accession","Experiment target","Biosample term name"]].to_csv("../data/encode.histone.tsv",sep="\t",index=None)


## Get TF metadata

In [42]:
metadata=pd.read_csv("../encode-tf/metadata.tsv",delimiter="\t")
print(metadata.shape)

(1357, 55)


In [43]:
metadata["Output type"].value_counts()

peaks and background as input for IDR     621
optimal IDR thresholded peaks             291
peaks                                     180
pseudoreplicated IDR thresholded peaks    141
conservative IDR thresholded peaks        124
Name: Output type, dtype: int64

In [44]:
metadata_stable=metadata[metadata["Output type"].isin(["optimal IDR thresholded peaks","conservative IDR thresholded peaks"])]

In [45]:
metadata_stable_noaudits=metadata_stable[metadata_stable["Audit ERROR"].isnull()]

In [46]:
df_exp=metadata_stable_noaudits[["Biosample term name","Experiment target"]].groupby(["Biosample term name","Experiment target"]).size().reset_index()

In [47]:
len(df_exp["Biosample term name"].unique())

84

In [48]:
len(df_exp["Experiment target"].unique())

12

In [49]:
df_exp[0].sum()

323

In [50]:
def create_combine_script(metadata):
    new_script_file = "/cellar/users/mpagadal/Data/projects/germline-immune/epigenetic/scripts/combine-bed-no-gz.sh"
    with open(new_script_file, 'w') as out_file:
        # header 
        out_file.write('#! /bin/bash\n')
        out_file.write('#SBATCH --mem=10G\n')
        out_file.write('#SBATCH -o ./out/%A.%x.%a.out # STDOUT\n')
        out_file.write('#SBATCH -e ./err/%A.%x.%a.err # STDERR\n')
        out_file.write("\n")
        # list of genes
        out_file.write("date\n")
        out_file.write("\n")
        for x in metadata["Biosample term name"].unique():
            df=metadata[metadata["Biosample term name"]==x]
            for y in df["Experiment target"].unique():
                files=df[df["Experiment target"]==y]["File accession"].tolist()
                files=["../encode-tf/"+x+".bed" for x in files]
                x=x.replace(" ","")
                x=x.replace("'","")
                x=x.replace(",","")
                y=y.split("-")[0]
                y=y.replace(" ","")
                out_file.write("cat {} > ../combined-beds/tf/{}.{}.bed".format(" ".join(files),x,y))
                out_file.write("\n")
                out_file.write("\n")
        
        out_file.write("date\n")


In [51]:
create_combine_script(metadata_stable_noaudits)

In [52]:
metadata_stable_noaudits[["Experiment accession","File accession","Experiment target","Biosample term name"]].to_csv("../data/encode.tf.tsv",sep="\t",index=None)